### YOLO to Standard format conversion script

In [ ]:
import cv2
import os
from tqdm import tqdm

base_dir = os.getcwd()

# Paths to the original dataset
ORIGINAL_TRAIN_DIR = os.path.join(base_dir,'dataset/train/images')
ORIGINAL_VAL_DIR = os.path.join(base_dir,'dataset/valid/images')
ORIGINAL_TEST_DIR = os.path.join(base_dir,'dataset/test/images')
TRAIN_LABELS_DIR = os.path.join(base_dir,'dataset/train/labels')
VAL_LABELS_DIR = os.path.join(base_dir,'dataset/valid/labels')
TEST_LABELS_DIR = os.path.join(base_dir,'dataset/test/labels')

# Paths to the cropped images based on labels
CROPPED_TRAIN_DIR = os.path.join(base_dir,'cropped_dataset/train')
CROPPED_VAL_DIR = os.path.join(base_dir,'cropped_dataset/valid')
CROPPED_TEST_DIR = os.path.join(base_dir,'cropped_dataset/test')

def preprocess_dataset(images_dir, labels_dir, cropped_dir):
    if not os.path.exists(cropped_dir):
        os.makedirs(cropped_dir)

    for label in ['awake', 'sleepy']:
        label_dir = os.path.join(cropped_dir, label)
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)

    for img_name in tqdm(os.listdir(images_dir), desc=f'Processing images in {images_dir}'):
        img_path = os.path.join(images_dir, img_name)
        label_path = os.path.join(labels_dir, os.path.splitext(img_name)[0] + '.txt')

        if not os.path.exists(label_path):
            continue

        with open(label_path, 'r') as f:
            labels = f.readlines()

        image = cv2.imread(img_path)
        height, width, _ = image.shape

        for label in labels:
            if label.strip() == '':
                continue

            label_parts = label.strip().split()
            if len(label_parts) != 5:
                continue

            class_id, x_center, y_center, bbox_width, bbox_height = map(float, label_parts)

            # Convert to pixel coordinates
            x1 = int((x_center - bbox_width / 2) * width)
            y1 = int((y_center - bbox_height / 2) * height)
            x2 = int((x_center + bbox_width / 2) * width)
            y2 = int((y_center + bbox_height / 2) * height)

            # Ensure bounding box coordinates are within image boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(width, x2)
            y2 = min(height, y2)

            # Crop the face region
            face_img = image[y1:y2, x1:x2]

            # Determine the label for saving the cropped face
            if int(class_id) == 0:
                face_label_dir = os.path.join(cropped_dir, 'awake')
            else:
                face_label_dir = os.path.join(cropped_dir, 'sleepy')

            # Save the cropped face image
            face_img_path = os.path.join(face_label_dir, img_name)
            cv2.imwrite(face_img_path, face_img)

In [ ]:
# Preprocess the datasets
preprocess_dataset(ORIGINAL_TRAIN_DIR, TRAIN_LABELS_DIR, CROPPED_TRAIN_DIR)
preprocess_dataset(ORIGINAL_VAL_DIR, VAL_LABELS_DIR, CROPPED_VAL_DIR)
preprocess_dataset(ORIGINAL_TEST_DIR, TEST_LABELS_DIR, CROPPED_TEST_DIR)
